In [1]:
!pip install emoji
import nltk
nltk.download('punkt')

     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=42f58474a15cc67fc93ce71ec31c03fb274b8beecbe5528da4a1b94646bd1e65
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from textblob import TextBlob
from bs4 import BeautifulSoup
import matplotlib.cm as cm
from string import punctuation
from string import digits
import urllib.request
import networkx as nx
import pandas as pd
import itertools
import emoji
import math
import time
import json
import sys
import re

# TODO: remove this when NetworkX is fixed
from warnings import simplefilter
import matplotlib.cbook
simplefilter("ignore", matplotlib.cbook.mplDeprecation)

In [3]:
# Import tweets
tweets = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/tweets/statuses.csv')

tweets.head()

,id,tweet_id,created_at,text,favorite_count,retweet_count,phone,sensitive,hashtags,no_hashtags,mentions,no_mentions,no_urls,no_media
0,113127283,1172524340002398208,2019-09-13 14:56:01,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
1,113127283,1172523885654355968,2019-09-13 14:54:13,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,2,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",5,0,1
2,113127283,1172523505537179648,2019-09-13 14:52:42,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,1,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
3,113127283,1172522933601886208,2019-09-13 14:50:26,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",6,0,1
4,113127283,1172522449184968705,2019-09-13 14:48:30,@enriqueserrano3 @Restrebooks @ELTIEMPO @enriq...,0,0,Twitter Web App,False,-,0,"[238261607, 3252651209, 9633802, 1073577478566...",4,0,1


In [0]:
# Import spanish stop word dictionary
url_sw = 'https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/stopwords-es.json'
response_sw = urllib.request.urlopen(url_sw)
data_sw = response_sw.read()

stop_words = set(json.loads(data_sw))

# Import emoji meanings
emoji_translations = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/utils/emojis_translated.csv')

# Get spanish meaning of an emoji
def emoji_meaning(emoji):
  meaning = emoji_translations.loc[emoji_translations['emoji'] == emoji]['translation']
  return meaning.values[0] if not meaning.empty else ''

# Transformations to remove digits and punctuation
remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans('', '', punctuation)

In [0]:
""" Processes text data in tweets
    
    text: text of tweet
    word_counts: word count dictionary
    
    returns processed text, length of content, polarity, subjectivity
"""
def process_text(text, word_counts):
    # Remove links
    text = ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())
    
    # Remove file names
    
        
    # Remove mentions
    text = ' '.join(re.sub("(@[A-Za-z0-9^\w]+)", " ", text.replace('@ ','@').replace('# ','#')).split())
        
    # Replace hashtags with words
    if text.count('#')>0:
        text = ' '.join(re.findall('[A-Z][^A-Z]*', text.replace('#', ' ')))
            
    #Remove HTML tags
    text = BeautifulSoup(text).get_text()
    
    # Save content length (exluding links and mentions)
    length = len(text)
        
    # Remove punctuation symbols
    text = ' '.join(re.sub("[\.\,\¡\¿\!\?\:\;\-\=\*\(\)\[\]\"\'\“\_\+\”\%\/\‘\’]", " ", text).split())
    text = text.translate(remove_digits).translate(remove_punctuation)
        
    # Lower case to avoid case sensitive problems
    text = text.lower()
        
    # Replace emojis with names 
    text = emoji.demojize(text)
        
    # Add space between emojis and other characters
    ind = -2
    for c in range(text.count(':')):
        ind = text.find(':',ind+2)
        if c%2==0:
            newLetter = ' :'
        else:
            newLetter = ': '
        text ="".join((text[:ind],newLetter,text[ind+1:]))
            
    # Replace emoji names with spanish meaning
    result = []
    parts = text.split(' ')
    for part in parts:
        if part:
            if part[0]==':':
                em = handle_emoji_tone(part)
                em = emoji_meaning(em)
                if em:
                    result.append(em)
            else:
                if part not in stop_words:
                    if part not in word_counts:
                        word_counts[part] = 1
                    else:
                        word_counts[part] += 1
                result.append(part)
        
    text = ' '.join(result)
        
    # Filter using NLTK library append it to a string
    word_tokens = word_tokenize(text)
        
    result = [w for w in word_tokens if not w in stop_words]
        
    text = ' '.join(result)
    
    if len(text)>12:
    
      analysis = TextBlob(text)

      try:
          eng = analysis.translate(to='en')
          sentiment = eng.sentiment     
          polarity = sentiment.polarity
          subjectivity = sentiment.subjectivity
          time.sleep(.5)
      except:
          print('Error')
          polarity = 0.0
          subjectivity = 0.0
    
    else:
      polarity = 0.0
      subjectivity = 0.0
    

    return text, length, polarity, subjectivity

tones = ['_light_skin_tone','_medium-light_skin_tone','_medium_skin_tone', 
             '_medium-dark_skin_tone','_dark_skin_tone']

# Method that removes the tone from emojis
def handle_emoji_tone(emoji):  
    for t in tones:
        if t in emoji:
            tone = t
            return emoji.replace(tone,'')
    return emoji


In [6]:
test = '@Tolaymaruja Las esperamos en el cine este fin de semana de estreno de @monosfilm #MONOS las abraza. https://t.co/L5B7xs9M3e '
process_text(test,{})

('esperamos cine semana estreno abraza', 81, 0.0, 0.0)

In [0]:
# Import the network
G = nx.read_graphml("network1.graphml")

In [31]:
''' Text analysis '''

# List to hold part of the dataset
text_analysis = []
text_analysis.append(['id', 'tweet_id','engagement',
            'effective_length', 'polarity', 'subjectivity'])

# List to store average engagement
avg_eng = []
avg_eng.append(['id', 'average_engagement', 'common_words', 'common_ht_words',
               'common_hts'])

#for n in G.nodes():
for n in list(G.nodes()):
    user_id = int(float(n))
    
    print ('User: ', user_id)
    
    # Total engagement
    eng = 0
    tweet_count = 0
    
    # Frequent word dictionary
    freq_words = {}

    # Frequent hashtag dictionary
    freq_hashtags = {}

    # Frequent words in hashtags dictionary
    freq_ht_words = {}
    
    user_tweets = tweets.loc[tweets['id'] == user_id]
    for index, row in user_tweets.iterrows():
      
        # Add current engagement
        currEng = int(row['favorite_count'])+int(row['retweet_count'])
        eng += currEng
        tweet_count +=1
        
        # Get hashtags
        ht = row['hashtags'].split(';') if row['hashtags'] != '-' else []
        
        # Update hashtag count
        for hashtag in ht:
            if hashtag not in freq_hashtags:
                freq_hashtags[hashtag] = 1
            else:
                freq_hashtags[hashtag] += 1
        
        # Separate hashtags by capitalization
        ht_words = [re.findall('[a-zA-Z][^A-Z]*', w) for w in ht]
        
        # Create a list with all the words in the hashtags
        hts = []
        for h in ht_words:
            hts = hts + h
        
        # Remove digits and lower caps for every hashtag word
        hts = [item.translate(remove_digits).lower() for item in hts]
        
        # Update hashtag word frequencies
        for word in hts:
            if word not in stop_words:
                if word not in freq_ht_words:
                    freq_ht_words[word] = 1
                else:
                    freq_ht_words[word] += 1
        
        text, length, polarity, subjectivity = process_text(row['text'], freq_words)
        
        # Update text in  data frame
        text_analysis.append([n, row['tweet_id'], currEng, length, polarity, subjectivity])
        print('Original:',row['text'],'\nProcessed:',text,'\nPolarity, subjectivity:',
              polarity, subjectivity,'\n')
    
    if tweet_count > 0:
      average_engagement = eng / tweet_count
    else:
      average_engagement = 0
    c_words = Counter(freq_words)
    c_ht_words = Counter(freq_ht_words)
    c_hashtags = Counter(freq_hashtags)
    avg_eng.append([user_id, average_engagement, c_words.most_common(3),
                   c_ht_words.most_common(3), c_hashtags.most_common(3)])

User:  426146744
Original: Partido de la selección, cervecita, amigos, y de remate Pipe Bueno y Paola Jara ¿nos vemos allá? #EfectyElGiroOficial https://t.co/AWVIjpwXLb 
Processed: partido selección cervecita amigos remate pipe paola jara vemos allá efecty giro oficial 
Polarity, subjectivity: 0.6 0.8 

Original: Hay gente descarada en la vida y este par. Qué gonorrea parce. https://t.co/WndKKPV8sH 
Processed: gente descarada vida par gonorrea parce 
Polarity, subjectivity: 0.0 0.0 

Original: No olvidemos JAJAJAA https://t.co/aOQHnIqA03 
Processed: olvidemos jajajaa 
Polarity, subjectivity: 0.2 0.4 

Original: Paaarce, me encanta como @CervezaAguila premia a los verdaderos #Creyentes de la selección. Esto lo motiva más a uno para seguir apoyando a la tricolor en cada partido ¡VAMOS COLOMBIA! https://t.co/LdgP4i0PQL 
Processed: paaarce encanta premia verdaderos creyentes selección motiva seguir apoyando tricolor partido 
Polarity, subjectivity: 0.3666666666666667 0.5 

Original: Si van

KeyError: ignored

In [17]:
# Save tweet analysis in a dataframe
analysis_results = pd.DataFrame(text_analysis[1:])
analysis_results.columns = text_analysis[0]
analysis_results["id"] = pd.to_numeric(analysis_results["id"])
analysis_results.to_csv('analysis_results.csv', index=False)
analysis_results.head()

,id,tweet_id,engagement,effective_length,polarity,subjectivity
0,272735394,1064288884484448258,15,1,-0.25,0.75
1,272735394,1060909398967500800,4,75,0.20,0.25
2,272735394,1055858907103588353,17,3,0.00,0.00
3,272735394,1050456122593828865,19,22,0.00,0.00
4,272735394,1048363465763696640,4,54,0.00,0.00


In [13]:
user_network_data=pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/user_network_data.csv')
user_network_data.head()

,id,katz,betweenness,authority
0,426146744.0,1.580982,0.000668,0.000029
1,162926902.0,3.461433,0.005760,0.000456
2,167435593.0,4.891715,0.003585,0.000369
3,269039712.0,1.375570,0.000193,0.000023
4,62561348.0,2.050953,0.000549,0.000141


In [26]:
# Create vertex data frame
vertices = pd.read_csv('https://raw.githubusercontent.com/cpenalozag/twitter_network/master/network-data/vertices.csv', encoding='utf-8', engine='c')
vertices["id"] = pd.to_numeric(vertices["id"])
vertices = vertices.drop(['description'], axis=1)
vertices.head()

,id,screen_name,followers,friends,verified,created_at,listed,tipo
0,426146744.0,SteevenOrozco,655637,476,False,2011,842,opinion
1,162926902.0,VivaAirCol,210761,30037,False,2010,475,empresa
2,167435593.0,Dimayor,1314384,147,True,2010,1308,competencia
3,269039712.0,cecorrehumor,156975,392,False,2011,183,comediante
4,62561348.0,ReddsColombia,25319,2517,True,2009,62,empresa


In [27]:
# Merge all user info

user_engagement = pd.DataFrame(avg_eng[1:])
user_engagement.columns = avg_eng[0]
user_engagement.head()

user_info = pd.merge(user_network_data, vertices, on='id')
user_info = pd.merge(user_info, user_engagement, on='id')
user_info.head()


,id,katz,betweenness,authority,screen_name,followers,friends,verified,created_at,listed,tipo,average_engagement,common_words,common_ht_words,common_hts
0,2.727354e+08,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF..."
1,4.255288e+09,1.044593,0.000003,5.229694e-07,Diana_GomezYN,49578,15,False,2015,33,opinion,418.500000,"[(señal, 1), (unimos, 1), (hundimos, 1)]","[(unimos, 1), (hundimos, 1)]","[(ONosUnimosONosHundimos, 1)]"


In [0]:
user_info.to_csv('user_info.csv', index=False)

In [29]:
# Merge user and tweet data
processed_data = pd.merge(user_info, analysis_results, on='id')
processed_data.head(n=10)

,id,katz,betweenness,authority,screen_name,followers,friends,verified,created_at,listed,tipo,average_engagement,common_words,common_ht_words,common_hts,tweet_id,engagement,effective_length,polarity,subjectivity
0,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1064288884484448258,15,1,-0.25,0.75
1,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1060909398967500800,4,75,0.20,0.25
2,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1055858907103588353,17,3,0.00,0.00
3,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1050456122593828865,19,22,0.00,0.00
4,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1048363465763696640,4,54,0.00,0.00
5,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1046894201387065349,24,7,0.00,0.00
6,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1041820147600769024,23,17,0.00,0.00
7,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1039746137903771649,8,47,0.00,0.00
8,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1039733623438680066,23,17,0.00,0.00
9,272735394.0,1.044593,0.000063,5.229694e-07,JuanEsteban_A,31112,12211,False,2011,41,opinion,12.779661,"[(canal, 5), (video, 3), (youtube, 3)]","[(foto, 2), (perfil, 2), (vamosmiseleccion, 1)]","[(Vamosmiseleccion, 1), (Colombia, 1), (NuevaF...",1036026128375402500,6,67,0.00,0.00


In [0]:
# Write processed data to file
processed_data.to_csv('processed_data.csv', index=False)